In [1]:
!pip install np_utils

In [4]:
from music21 import *
import keras
from tensorflow.keras import backend as k
import glob
import numpy
import np_utils
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation,Dropout,LSTM
from tensorflow.keras.optimizers import RMSprop
import librosa

In [5]:
notes = []
counter = 0

new_directory_path = "midi_songs/*.mid"

for file in glob.glob(new_directory_path):
    print('processing song no: ', counter)
    counter = counter + 1
    midi = converter.parse(file)
    notes_to_parse = None
    parts = instrument.partitionByInstrument(midi)
    
    if parts:  # file has instrument parts
        notes_to_parse = parts.parts[0].recurse()
    else:  # file has notes in a flat structure
        notes_to_parse = midi.flat.notes
    
    for element in notes_to_parse:
        if isinstance(element, note.Note):
            notes.append(str(element.pitch))
        elif isinstance(element, chord.Chord):
            notes.append('.'.join(str(n) for n in element.normalOrder))


processing song no:  0
processing song no:  1
processing song no:  2
processing song no:  3
processing song no:  4
processing song no:  5
processing song no:  6
processing song no:  7
processing song no:  8
processing song no:  9
processing song no:  10
processing song no:  11
processing song no:  12


/Users/Aryan/mambaforge/envs/Lofi_Vibe_Generator/lib/python3.11/site-packages/music21/midi/translate.py:874: TranslateWarning: Unable to determine instrument from <music21.midi.MidiEvent SEQUENCE_TRACK_NAME, track=5, channel=None, data=b'Martin Rosok Copyright \xa9 1998'>; getting generic Instrument
  warnings.warn(


processing song no:  13
processing song no:  14
processing song no:  15
processing song no:  16
processing song no:  17
processing song no:  18
processing song no:  19
processing song no:  20
processing song no:  21
processing song no:  22
processing song no:  23
processing song no:  24
processing song no:  25
processing song no:  26
processing song no:  27
processing song no:  28
processing song no:  29
processing song no:  30
processing song no:  31
processing song no:  32
processing song no:  33
processing song no:  34
processing song no:  35
processing song no:  36
processing song no:  37
processing song no:  38
processing song no:  39
processing song no:  40
processing song no:  41
processing song no:  42
processing song no:  43
processing song no:  44
processing song no:  45
processing song no:  46
processing song no:  47
processing song no:  48
processing song no:  49
processing song no:  50
processing song no:  51
processing song no:  52


/Users/Aryan/mambaforge/envs/Lofi_Vibe_Generator/lib/python3.11/site-packages/music21/midi/translate.py:874: TranslateWarning: Unable to determine instrument from <music21.midi.MidiEvent SEQUENCE_TRACK_NAME, track=6, channel=None, data=b'Pok\xe9mon: Elite Four (Piano)'>; getting generic Instrument
  warnings.warn(


processing song no:  53
processing song no:  54
processing song no:  55
processing song no:  56
processing song no:  57
processing song no:  58
processing song no:  59
processing song no:  60
processing song no:  61
processing song no:  62
processing song no:  63
processing song no:  64
processing song no:  65
processing song no:  66
processing song no:  67
processing song no:  68
processing song no:  69
processing song no:  70
processing song no:  71
processing song no:  72
processing song no:  73
processing song no:  74
processing song no:  75
processing song no:  76
processing song no:  77
processing song no:  78
processing song no:  79
processing song no:  80
processing song no:  81
processing song no:  82
processing song no:  83
processing song no:  84
processing song no:  85
processing song no:  86
processing song no:  87
processing song no:  88
processing song no:  89
processing song no:  90
processing song no:  91


In [6]:
sequence_length = 100
n_vocab = len(set(notes))

# get all pitch names
pitchnames = sorted(set(item for item in notes))
# create a dictionary to map pitches to integers
note_to_int = dict((note, number) for number, note in enumerate(pitchnames))
network_input = []
network_output = []
# create input sequences and the corresponding outputs
for i in range(0, len(notes) - sequence_length, 1):
    sequence_in = notes[i:i + sequence_length]
    sequence_out = notes[i + sequence_length]
    network_input.append([note_to_int[char] for char in sequence_in])
    network_output.append(note_to_int[sequence_out])
n_patterns = len(network_input)
# reshape the input into a format compatible with LSTM layers
network_input = numpy.reshape(network_input, (n_patterns, sequence_length, 1))
# normalize input
network_input = network_input / float(n_vocab)
print(network_input)
# Check if network_output is not empty
if not network_output:
    raise ValueError("The network_output list is empty. Check your data.")

network_output = keras.utils.to_categorical(network_output, num_classes=n_vocab)

# ... rest of your code

# If this is within a function, you might return the processed data
# return network_input, network_output


[[[0.95092025]
  [0.95092025]
  [0.8006135 ]
  ...
  [0.94785276]
  [0.95705521]
  [0.99386503]]

 [[0.95092025]
  [0.8006135 ]
  [0.95092025]
  ...
  [0.95705521]
  [0.99386503]
  [0.97546012]]

 [[0.8006135 ]
  [0.95092025]
  [0.99079755]
  ...
  [0.99386503]
  [0.97546012]
  [0.94785276]]

 ...

 [[0.97239264]
  [0.8006135 ]
  [0.97239264]
  ...
  [0.33128834]
  [0.86196319]
  [0.94785276]]

 [[0.8006135 ]
  [0.97239264]
  [0.99079755]
  ...
  [0.86196319]
  [0.94785276]
  [0.33128834]]

 [[0.97239264]
  [0.99079755]
  [0.95398773]
  ...
  [0.94785276]
  [0.33128834]
  [0.86196319]]]


In [7]:
model = Sequential()
model.add(LSTM(
        256,
        input_shape=(network_input.shape[1], network_input.shape[2]),
        return_sequences=True
    ))
model.add(Dropout(0.3))
model.add(LSTM(512, return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(256))
model.add(Dense(256))
model.add(Dropout(0.3))
model.add(Dense(n_vocab))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

In [8]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 100, 256)          264192    
                                                                 
 dropout (Dropout)           (None, 100, 256)          0         
                                                                 
 lstm_1 (LSTM)               (None, 100, 512)          1574912   
                                                                 
 dropout_1 (Dropout)         (None, 100, 512)          0         
                                                                 
 lstm_2 (LSTM)               (None, 256)               787456    
                                                                 
 dense (Dense)               (None, 256)               65792     
                                                                 
 dropout_2 (Dropout)         (None, 256)               0

In [ ]:
filepath = "weights-improvement-{epoch:02d}-{loss:.4f}-bigger.hdf5"
checkpoint = keras.callbacks.ModelCheckpoint(
    filepath, monitor='loss',
    verbose=0,
    save_best_only=True,
    mode='min'
)
callbacks_list = [checkpoint]
model.fit(network_input, network_output, epochs=200, batch_size=512, callbacks=callbacks_list)

Epoch 1/200
90/90 [==============================] - 340s 4s/step - loss: 4.7860
Epoch 2/200


/Users/Aryan/mambaforge/envs/Lofi_Vibe_Generator/lib/python3.11/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


22/90 [======>.......................] - ETA: 5:00 - loss: 4.7089

In [ ]:
import time
start = numpy.random.randint(0, len(network_input)-1)
int_to_note = dict((number, note) for number, note in enumerate(pitchnames))
pattern = network_input[start]

prediction_output = []
offset = 0
output_notes = []
while True:
    prediction_input = numpy.reshape(pattern, (1, len(pattern), 1))
    prediction_input = prediction_input / float(n_vocab)
    prediction = model.predict(prediction_input, verbose=0)
    index = numpy.argmax(prediction)
    try:
      result = int_to_note[index]
    except KeyError:
    # Handle out-of-range index, for example:
      result = "C4"  # Default note

    prediction_output.append(result)
    ind =numpy.asarray([index])
    pattern = numpy.append(pattern, ind)
    pattern = pattern[1:len(pattern)]

    if ('.' in result) or result.isdigit():
        notes_in_chord = result.split('.')
        notes = []
        for current_note in notes_in_chord:
            new_note = note.Note(int(current_note))
            new_note.storedInstrument = instrument.Piano()
            notes.append(new_note)
        new_chord = chord.Chord(notes)
        new_chord.offset = offset
        output_notes.append(new_chord)
    # pattern is a note
    else:
        rounded_pitch = round(pattern[0])
        if rounded_pitch in int_to_note and '.' not in int_to_note[rounded_pitch] and '' not in int_to_note[rounded_pitch]:
            new_note = note.Note(int_to_note[rounded_pitch])
            new_note.offset = offset
            new_note.storedInstrument = instrument.Piano()
            output_notes.append(new_note)
    # increase offset each iteration so that notes do not stack
    offset += 0.5

    midi_stream = stream.Stream(output_notes)
    midi_stream.write('midi', fp='test_output.mid')

    # Play the generated music (you may need to adjust the player based on your environment)
    midi_stream.show('midi')

    # Pause for a short duration before the next iteration
    time.sleep(1)

In [ ]:
from music21 import midi, stream

# Create a Score object
s = stream.Score()

# Load MIDI file into Score
mf = midi.MidiFile()
mf.open('test_output.mid')
mf.read()
mf.close()
s.append(midi.translate.midiFileToStream(mf))

# Show the MIDI stream
s.show('midi')